# Import

In [54]:
%pip install matplotlib
%pip install fuzzywuzzy
import json
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
print(matplotlib.__version__)
from fuzzywuzzy import fuzz

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
3.7.1


# Data

In [55]:
letras = pd.read_csv('letras_preproces.csv')
audio = pd.read_csv('featuresLow_data.csv')

In [56]:
# LETRAS SONG ID
songId = []
for k, song in enumerate(letras["Letra"]):
	aux = song.split("-", maxsplit=1)[-1]
	aux = aux.strip().lower()
	aux = aux.replace('none-','').replace("'","").replace("...","").replace("the-","").replace("(feat.-in-flames)-","").replace("---remastered","")
	songId.append(aux)
letras["songId"] = songId

In [57]:
# AUDIO SONG ID
songId = []
for k, song in enumerate(audio["track_uri"]):
	
	aux = str(audio["track_name"][k]).replace(" ","-").replace("/","-").replace("'","").lower() \
    + "-" + str(audio["artist_name"][k]).replace(" ","-").replace("/","-").replace("'","").lower() \
    + ".txt"

	aux = aux.replace("---main","").replace("sehen-","").replace("...","").replace("the-","").replace("(feat.-in-flames)-","").replace("---remastered","")
	songId.append(aux)
audio["songId"] = songId

In [58]:
# Utilizar fuzzywuzzy para encontrar coincidencias cercanas
def fuzzy_merge(df1, df2, key1, key2, threshold=90, limit=2):
    """
    Realiza una fusión entre dos dataframes utilizando la librería fuzzywuzzy para encontrar las coincidencias más cercanas.
    """
    s = df2[key2].tolist()

    m = df1[key1].apply(lambda x: pd.Series([fuzz.token_set_ratio(x, i) for i in s if isinstance(i, str)]).nlargest(limit))

    df1['matches'] = m.apply(lambda x: ', '.join(s[x.index[x >= threshold]].tolist()), axis=1)

    return pd.merge(df1, df2, left_on='matches', right_on=key2, how='left')



In [59]:
# Realizar la fusión utilizando los identificadores únicos
#merged_df = fuzzy_merge(letras, audio, 'songId', 'songId')


In [60]:
df = pd.merge(letras, audio, on='songId', how='outer')
df


,Letra,Idioma,lyrics_state,hasLetra,Lyrics,Lyrics_proces,songId,track_uri,track_name,artist_uri,...,liveness,valence,tempo,id_song,uri,track_href,analysis_url,duration_ms,time_signature,dataCancion
0,Data/Letras/1-'Till-I-Collapse-Eminem.txt,en,complete,1.0,TranslationsEspañolTürkçePortuguêsItalianoहिन्...,cause feel tired left left feel weak feel weak...,till-i-collapse-eminem.txt,spotify:track:4xkOaSrkexMciUUogZKVTS,Till I Collapse,spotify:artist:7dGJo4pcD2V6oG8kP0tJRR,...,0.0816,0.1000,171.447,4xkOaSrkexMciUUogZKVTS,spotify:track:4xkOaSrkexMciUUogZKVTS,https://api.spotify.com/v1/tracks/4xkOaSrkexMc...,https://api.spotify.com/v1/audio-analysis/4xkO...,297787.0,4.0,"{'danceability': 0.548, 'energy': 0.847, 'key'..."
1,Data/Letras/2-St.-Anger-Metallica.txt,en,complete,1.0,St. Anger Lyrics[Verse]\nSaint Anger 'round my...,saint anger round neck saint anger round neck ...,st.-anger-metallica.txt,spotify:track:3fOc9x06lKJBhz435mInlH,St. Anger,spotify:artist:2ye2Wgw4gimLv2eAKyk1NB,...,0.0953,0.4980,185.252,3fOc9x06lKJBhz435mInlH,spotify:track:3fOc9x06lKJBhz435mInlH,https://api.spotify.com/v1/tracks/3fOc9x06lKJB...,https://api.spotify.com/v1/audio-analysis/3fOc...,441133.0,4.0,"{'danceability': 0.249, 'energy': 0.949, 'key'..."
2,Data/Letras/3-Speedin'-Rick-Ross.txt,en,complete,1.0,Speedin’ Lyrics[Intro: Rick Ross]\nLegendary\n...,legendary runners know trilla dollar count acc...,speedin-rick-ross.txt,spotify:track:3Y96xd4Ce0J47dcalLrEC8,Speedin',spotify:artist:1sBkRIssrMs1AbVkOJbc7a,...,0.2100,0.4780,100.059,3Y96xd4Ce0J47dcalLrEC8,spotify:track:3Y96xd4Ce0J47dcalLrEC8,https://api.spotify.com/v1/tracks/3Y96xd4Ce0J4...,https://api.spotify.com/v1/audio-analysis/3Y96...,204960.0,4.0,"{'danceability': 0.668, 'energy': 0.787, 'key'..."
3,Data/Letras/4-Bamboo-Banga-M.I.A..txt,en,complete,1.0,"Bamboo Banga Lyrics[Intro]\nRoad runner, road ...",road runner road runner going miles hour road ...,bamboo-banga-m.i.a..txt,spotify:track:6tqFC1DIOphJkCwrjVzPmg,Bamboo Banga,spotify:artist:0QJIPDAEDILuo8AIq3pMuU,...,0.0691,0.7130,125.984,6tqFC1DIOphJkCwrjVzPmg,spotify:track:6tqFC1DIOphJkCwrjVzPmg,https://api.spotify.com/v1/tracks/6tqFC1DIOphJ...,https://api.spotify.com/v1/audio-analysis/6tqF...,298360.0,4.0,"{'danceability': 0.805, 'energy': 0.918, 'key'..."
4,Data/Letras/5-Die-MF-Die-Dope.txt,en,complete,1.0,Die MF Die Lyrics[Intro]\nDie!\n\n[Verse 1]\nI...,die need forgiveness need hate need acceptance...,die-mf-die-dope.txt,spotify:track:5bU4KX47KqtDKKaLM4QCzh,Die MF Die,spotify:artist:7fWgqc4HJi3pcHhK8hKg2p,...,0.1090,0.5670,126.020,5bU4KX47KqtDKKaLM4QCzh,spotify:track:5bU4KX47KqtDKKaLM4QCzh,https://api.spotify.com/v1/tracks/5bU4KX47KqtD...,https://api.spotify.com/v1/audio-analysis/5bU4...,186067.0,4.0,"{'danceability': 0.657, 'energy': 0.96, 'key':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74530,NaN,NaN,NaN,NaN,NaN,NaN,borcun-yok-harika-avcı.txt,spotify:track:4zANGx3Z3kZzrYcXftaTpt,Borcun Yok,spotify:artist:7kSXfKnz2ZHGNyBN1PF7FV,...,0.1110,0.5210,148.005,4zANGx3Z3kZzrYcXftaTpt,spotify:track:4zANGx3Z3kZzrYcXftaTpt,https://api.spotify.com/v1/tracks/4zANGx3Z3kZz...,https://api.spotify.com/v1/audio-analysis/4zAN...,303133.0,4.0,"{'danceability': 0.573, 'energy': 0.597, 'key'..."
74531,NaN,NaN,NaN,NaN,NaN,NaN,breath-journey---breath-of-god-+-takahiro-kido...,spotify:track:7ypbfg0NMXz5OIWSIxoZBw,Breath Journey - Breath of God + Takahiro Kido...,spotify:artist:36lwvXwVvWdQokNoCZH0JK,...,0.0961,0.0361,83.617,7ypbfg0NMXz5OIWSIxoZBw,spotify:track:7ypbfg0NMXz5OIWSIxoZBw,https://api.spotify.com/v1/tracks/7ypbfg0NMXz5...,https://api.spotify.com/v1/audio-analysis/7ypb...,533240.0,3.0,"{'danceability': 0.0937, 'energy': 0.35, 'key'..."
74532,NaN,NaN,NaN,NaN,NaN,NaN,sirrus-theme-–-mechanical-age-robyn-miller.txt,spotify:track:0WlqKXIDbE3qj4BNzmiECV,Sirrus' Theme – Mechanical Age,spotify:artist:70EWCtKhQhtClb7NWQlm0d,...,0.0901,0.3450,119.865,0WlqKXIDbE3qj4BNzmiECV,spotify:track:0WlqKXIDbE3qj4BNzmiECV,https://a

In [61]:
not_matched = df["track_uri"].isna()
#not_matched = df_merged.isna().any(axis=1)

# Mostrar los registros que no hicieron match
df[not_matched]


,Letra,Idioma,lyrics_state,hasLetra,Lyrics,Lyrics_proces,songId,track_uri,track_name,artist_uri,...,liveness,valence,tempo,id_song,uri,track_href,analysis_url,duration_ms,time_signature,dataCancion
91,Data/Letras/91-Come-to-Daddy-Aphex-Twin.txt,en,complete,1.0,DJ KamZ’s CD Collection Lyrics#6ix9ine - Day69...,ixine day graduation dayaabove group therapy a...,come-to-daddy-aphex-twin.txt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,Data/Letras/92-Fields-of-Desolation-Arch-Enemy...,en,complete,1.0,Fields of Desolation LyricsOn the fields of de...,fields desolation crying arms fear unknown fee...,fields-of-desolation-arch-enemy.txt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,Data/Letras/102-Itsari-Sepultura.txt,NaN,complete,1.0,Itsári Lyrics[Instrumental w/ chanting]You mig...,like,itsari-sepultura.txt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,Data/Letras/117-The-Rape-Over-Mos-Def.txt,en,complete,1.0,The Rape Over Lyrics[Intro]\nC'mon\nBoogie man...,cmon boogie man business listen old white men ...,rape-over-mos-def.txt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,Data/Letras/118-Black-Fire-Upon-Us-Dethklok.txt,en,complete,1.0,"Black Fire Upon Us Lyrics[Verse 1]\nTonight, w...",tonight ride clouds fire damned gods deaths co...,black-fire-upon-us-dethklok.txt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62333,Data/Letras/61609-Borcun-Yok-Harika-Avci.txt,tr,complete,1.0,1 ContributorBeyler Bayanlar LyricsIvır zıvıra...,ivr zvra taklp gidiyoruz bak ben zannederdim s...,borcun-yok-harika-avci.txt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62334,Data/Letras/61610-y-Takahiro-Kido.txt,pt,complete,1.0,42 ContributorsExplicando os Nomes Artísticos ...,achou alguma cantora rapper produtora dupla tr...,y-takahiro-kido.txt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62338,Data/Letras/61614-NONE-Mechanical-Age---Sirrus...,0,0,0.0,NaN,NaN,mechanical-age---sirrus-theme-robyn-miller.txt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62339,Data/Letras/61615-NONE-Stoneship-Age---Sirrus'...,0,0,0.0,NaN,NaN,stoneship-age---sirrus-theme-robyn-miller.txt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
print(audio.loc[89]["songId"])
print(letras.loc[89]["songId"])

archives-of-pain-manic-street-preachers.txt
archives-of-pain-manic-street-preachers.txt


In [65]:
df.dropna(subset=['liveness'], inplace=True)
df.dropna(subset=['Letra'], inplace=True)

df

,Letra,Idioma,lyrics_state,hasLetra,Lyrics,Lyrics_proces,songId,track_uri,track_name,artist_uri,...,liveness,valence,tempo,id_song,uri,track_href,analysis_url,duration_ms,time_signature,dataCancion
0,Data/Letras/1-'Till-I-Collapse-Eminem.txt,en,complete,1.0,TranslationsEspañolTürkçePortuguêsItalianoहिन्...,cause feel tired left left feel weak feel weak...,till-i-collapse-eminem.txt,spotify:track:4xkOaSrkexMciUUogZKVTS,Till I Collapse,spotify:artist:7dGJo4pcD2V6oG8kP0tJRR,...,0.0816,0.1000,171.447,4xkOaSrkexMciUUogZKVTS,spotify:track:4xkOaSrkexMciUUogZKVTS,https://api.spotify.com/v1/tracks/4xkOaSrkexMc...,https://api.spotify.com/v1/audio-analysis/4xkO...,297787.0,4.0,"{'danceability': 0.548, 'energy': 0.847, 'key'..."
1,Data/Letras/2-St.-Anger-Metallica.txt,en,complete,1.0,St. Anger Lyrics[Verse]\nSaint Anger 'round my...,saint anger round neck saint anger round neck ...,st.-anger-metallica.txt,spotify:track:3fOc9x06lKJBhz435mInlH,St. Anger,spotify:artist:2ye2Wgw4gimLv2eAKyk1NB,...,0.0953,0.4980,185.252,3fOc9x06lKJBhz435mInlH,spotify:track:3fOc9x06lKJBhz435mInlH,https://api.spotify.com/v1/tracks/3fOc9x06lKJB...,https://api.spotify.com/v1/audio-analysis/3fOc...,441133.0,4.0,"{'danceability': 0.249, 'energy': 0.949, 'key'..."
2,Data/Letras/3-Speedin'-Rick-Ross.txt,en,complete,1.0,Speedin’ Lyrics[Intro: Rick Ross]\nLegendary\n...,legendary runners know trilla dollar count acc...,speedin-rick-ross.txt,spotify:track:3Y96xd4Ce0J47dcalLrEC8,Speedin',spotify:artist:1sBkRIssrMs1AbVkOJbc7a,...,0.2100,0.4780,100.059,3Y96xd4Ce0J47dcalLrEC8,spotify:track:3Y96xd4Ce0J47dcalLrEC8,https://api.spotify.com/v1/tracks/3Y96xd4Ce0J4...,https://api.spotify.com/v1/audio-analysis/3Y96...,204960.0,4.0,"{'danceability': 0.668, 'energy': 0.787, 'key'..."
3,Data/Letras/4-Bamboo-Banga-M.I.A..txt,en,complete,1.0,"Bamboo Banga Lyrics[Intro]\nRoad runner, road ...",road runner road runner going miles hour road ...,bamboo-banga-m.i.a..txt,spotify:track:6tqFC1DIOphJkCwrjVzPmg,Bamboo Banga,spotify:artist:0QJIPDAEDILuo8AIq3pMuU,...,0.0691,0.7130,125.984,6tqFC1DIOphJkCwrjVzPmg,spotify:track:6tqFC1DIOphJkCwrjVzPmg,https://api.spotify.com/v1/tracks/6tqFC1DIOphJ...,https://api.spotify.com/v1/audio-analysis/6tqF...,298360.0,4.0,"{'danceability': 0.805, 'energy': 0.918, 'key'..."
4,Data/Letras/5-Die-MF-Die-Dope.txt,en,complete,1.0,Die MF Die Lyrics[Intro]\nDie!\n\n[Verse 1]\nI...,die need forgiveness need hate need acceptance...,die-mf-die-dope.txt,spotify:track:5bU4KX47KqtDKKaLM4QCzh,Die MF Die,spotify:artist:7fWgqc4HJi3pcHhK8hKg2p,...,0.1090,0.5670,126.020,5bU4KX47KqtDKKaLM4QCzh,spotify:track:5bU4KX47KqtDKKaLM4QCzh,https://api.spotify.com/v1/tracks/5bU4KX47KqtD...,https://api.spotify.com/v1/audio-analysis/5bU4...,186067.0,4.0,"{'danceability': 0.657, 'energy': 0.96, 'key':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62350,Data/Letras/61626-Secret-Quietdrive.txt,en,complete,1.0,"2 ContributorsSecret LyricsHey, bride of mine\...",hey bride love find love death true doctor ask...,secret-quietdrive.txt,spotify:track:2bRIsZ92JRKlvQOZlyR9CO,Secret,spotify:artist:5L1n6wVJMxBw6sZbxD0XAU,...,0.1350,0.0734,167.996,2bRIsZ92JRKlvQOZlyR9CO,spotify:track:2bRIsZ92JRKlvQOZlyR9CO,https://api.spotify.com/v1/tracks/2bRIsZ92JRKl...,https://api.spotify.com/v1/audio-analysis/2bRI...,258373.0,4.0,"{'danceability': 0.396, 'energy': 0.915, 'key'..."
62351,Data/Letras/61627-The-Last-of-the-Rest-Was-the...,en,complete,1.0,1 ContributorThe Last Of The Rest Was The End ...,desperate days solutions sold price high exit ...,last-of-rest-was-end-medications.txt,spotify:track:7o3Np7cho9cBCrNDokxzYC,The Last of the Rest Was the End,spotify:artist:6ZgHswsP4qcUXXtJepAIU2,...,0.0782,0.3400,144.844,7o3Np7cho9cBCrNDokxzYC,spotify:track:7o3Np7cho9cBCrNDokxzYC,https://api.spotify.com/v1/tracks/7o3Np7cho9cB...,https://api.spotify.com/v1/audio-analysis/7o3N...,324000.0,3.0,"{'danceability': 0.244, 'energy': 0.866, 'key'..."
62352,Data/Letras/61628-Lovechild-Daniel

In [66]:
df.to_csv('Data_unificada.csv', header=True, index=False)